In [109]:
# Numerical
from jax.numpy import exp, newaxis, mean   # Arithematic
from jax.numpy import array, zeros, arange, where # Array creation routines
from jax.numpy import float64
from jax.numpy import sum as vsum
from jax.numpy.linalg import solve, eig, norm
from jax.random import PRNGKey
from jax import jit, jacrev, jacfwd, vmap, grad, jvp, vjp

# Miscellaneous
from functools import partial 

# Hamiltonian monte-carlo
from numpyro.infer import NUTS, MCMC

# Visualisation
import matplotlib.pyplot as plt

In [110]:
def construct_analytic_solution():
    """
    This creates a closure environment that pre-calculates the eigenvalues of the transfer matrix and the coefficients that produce the approriate initial state.
    """
    cbm = load_presaved_model("Guttler14", production_rate_units="atoms/cm^2/s")
    cbm.compile()

    eigenvalues, eigenvectors = eig(cbm._matrix)    # Constructing the linear eigenfunctions
    eigenvalues = eigenvalues.real  # Making sure that values are real
    eigenvectors = eigenvectors.real    # Making sure that the eigenvectors are real 

    soln = solve(eigenvectors, cbm._production_coefficients)
    soln = 1 / norm(soln) * soln    # Normalising the impulse 
    soln = soln.real

    @partial(jit, static_argnums=(3, 4, 5))
    def analytic_solution(time, start, area, coeffs=soln, eigenvals=eigenvalues, eigenvecs=eigenvectors):
        """
        This is the analytic solution itself.
        
        Parameters: 
            time: Array -> The time values that the solution is to be evaluated at 
            coeffs: Array -> The coefficients associated with a particular initial condition. 
          
            eigenvals: Array -> The eigenvalues of the transfer matrix used to construct the linear superposition of the eigenfunctions
            eigenvecs: Array[Array] -> The eigenvectors of the transfer matrix used to construct the linear superposition of the eigenfunctions.
        
        Returns:
            Array -> The analytic solution evaluated at times
        """
        @jit
        def sum_util(coeff, eigenval, eigenvec, /, time=time, start=start, area=area):
            return area * coeff * exp(eigenval * (time - start))[:, newaxis] * eigenvec

        impulse = vsum(vmap(sum_util)(coeffs, eigenvals, eigenvecs.T), axis=0)
        return where(time.reshape(time.shape[0], 1) > start, impulse, zeros(impulse.shape))

    return analytic_solution

In [111]:
@partial(jit, static_argnums=(0))
def load(filename: str):
    """
    A custom `JAX` file loading protocol designed to be very quick and return a value that is `JAX` transformable. 
    
    Parameters:
        filename: String -> The file address of the data
    Returns:
        DeviceArray -> The data in column major order
    """
    with open(filename) as data:    # Opening the data file
        header = next(data)
        data = array([row.strip().split(" ") for row in data], dtype=float64)
        return data.T


In [112]:
analytic_solution = construct_analytic_solution()
data = load("miyake12.csv")
data = data.at[1].add(-mean(data[1, 1:4]))

In [113]:
def construct_loss(analytic_solution=analytic_solution, data=data):
    @jit
    def loss(parameters):
        start, area = parameters
        analytic_data = analytic_solution(data[0], start, area)
        return vsum((data[1] - analytic_data[:, 1]) ** 2 / data[2] ** 2)
    
    return loss

In [114]:
# Autodiff declarations for the model
loss = construct_loss()
gradient = jit(grad(loss))
hessian = jit(jacfwd(grad(loss)))

In [115]:
# Running the No U Turn sampling
nuts_kernel = NUTS(potential_fn=loss)
mcmc = MCMC(nuts_kernel, num_warmup=100, num_samples=500)
mcmc.run(random.PRNGKey(11), init_params=(774.86, 80.0))

sample: 100%|██████████| 600/600 [00:03<00:00, 180.72it/s, 3 steps of size 4.15e-01. acc. prob=0.87] 


In [117]:
test = mcmc.get_samples()

In [119]:
import seaborn as sns

In [120]:
sns.histplot(test[1])

AttributeError: module 'seaborn' has no attribute 'plothist'